In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
# from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from gensim.models import Doc2Vec
# from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing
import pandas as pd

# 9.2　特征提取

In [ ]:
def load_all_files():
    import glob
    x=[]
    y=[]
    #加载攻击样本
    files=glob.glob("../Data/ADFA-LD/Attack_Data_Master/*/*")
    for file in files:
        with open(file) as f:
            lines=f.readlines()
        x.append(" ".join(lines))
        y.append(1)
    print("Load black data %d" % len(x))
    
    #加载正常样本
    files=glob.glob("../Data/ADFA-LD/Training_Data_Master/*")
    for file in files:
        with open(file) as f:
            lines=f.readlines()
        x.append(" ".join(lines))
        y.append(0)
    print("Load full data %d" % len(x))

    return x,y
x,y = load_all_files()

Load black data 746
Load full data 1579


In [9]:
pd.DataFrame({'调用序列':x,'标签':y})

,调用序列,标签
0,265 168 168 265 168 168 168 265 168 265 168 16...,1
1,3 104 91 265 142 142 104 3 175 3 142 142 146 1...,1
2,265 43 3 265 43 168 265 168 168 168 3 43 168 3...,1
3,309 54 309 309 54 309 54 54 54 3 54 54 3 54 54...,1
4,168 3 168 168 168 265 265 265 168 168 265 168 ...,1
...,...,...
1574,11 45 33 192 33 5 197 192 6 33 5 3 197 192 192...,0
1575,11 45 33 192 33 5 197 192 6 33 5 3 197 192 192...,0
1576,4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 ...,0
1577,6 11 11 45 33 192 33 5 195 5 195 5 195 5 195 3...,0


In [10]:
def get_feature_wordbag():
    max_features=1000
    x,y=load_all_files()
    vectorizer = CountVectorizer(
                                 ngram_range=(3, 3), # 使用CountVectorizer进行N-Gram处理，这里N取3，即使用3-Gram
                                 token_pattern=r'\b\d+\b', # 单词切分使用的正则表达式
                                 decode_error='ignore',
                                 strip_accents='ascii',
                                 max_features=max_features,
                                 stop_words='english',
                                 max_df=1.0,
                                 min_df=1 )
    print(vectorizer)
    x = vectorizer.fit_transform(x)
    # N-Gram处理后进行TF-IDF处理，提升分类算法的性能：
    transformer = TfidfTransformer(smooth_idf=False)
    x=transformer.fit_transform(x)

    x = x.toarray()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_feature_wordbag()

Load black data 746
Load full data 1579
CountVectorizer(decode_error='ignore', max_features=1000, ngram_range=(3, 3),
                stop_words='english', strip_accents='ascii',
                token_pattern='\\b\\d+\\b')


In [12]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.072962,...,0.0,0.000000,0.081887,0.0,0.0,0.0,0.159212,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.117069,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.048005,0.029360,0.0,0.0,0.0,0.000000,0.029614,0.058474,0.055238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
943,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
944,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
945,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


# 9.3　模型训练与验证

## 9.3.1　朴素贝叶斯算法

In [13]:
def do_nb(x_train, x_test, y_train, y_test):
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

do_nb(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.87      0.98      0.92       339
           1       0.97      0.83      0.90       293

    accuracy                           0.91       632
   macro avg       0.92      0.91      0.91       632
weighted avg       0.92      0.91      0.91       632

[[332   7]
 [ 49 244]]


## 9.3.2　XGBoost算法

In [14]:

def do_xgboost(x_train, x_test, y_train, y_test):
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
do_xgboost(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       339
           1       0.95      0.96      0.96       293

    accuracy                           0.96       632
   macro avg       0.96      0.96      0.96       632
weighted avg       0.96      0.96      0.96       632

[[325  14]
 [ 12 281]]


## 9.3.3　深度学习算法之多层感知机

In [16]:
def do_mlp(x_train, x_test, y_train, y_test):
    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
do_mlp(x_train, x_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       339
           1       0.95      0.97      0.96       293

    accuracy                           0.96       632
   macro avg       0.96      0.96      0.96       632
weighted avg       0.96      0.96      0.96       632

[[324  15]
 [ 10 283]]
